In [1]:
import bs4
import pandas as pd
import requests
from io import StringIO
import time
import os
import random



In [3]:
import yfinance as yf

TICKER_FILE = r"C:\Users\surji\Desktop\Quant_Poject\Complete_analysis_200stocks\Data\Top200_StockAnalysis.xlsx"
TICKER_COLUMN = "Ticker"

SAVE_DIR = r"C:\Users\surji\Desktop\Quant_Poject\Complete_analysis_200stocks\Data\Files"
os.makedirs(SAVE_DIR, exist_ok=True)

tickers = pd.read_excel(TICKER_FILE)[TICKER_COLUMN].dropna().unique().tolist()

volume_data = []
marketcap_data = []
dividendyield_data = []

def get_historical_data(ticker, start='2010-01-01', end='2023-12-31'):
    try:
        stock = yf.Ticker(ticker)
        hist = stock.history(start=start, end=end, actions=True)

        shares_outstanding = stock.info.get('sharesOutstanding', None)
        hist['MarketCap'] = hist['Close'] * shares_outstanding if shares_outstanding else None

        hist['DividendYield'] = 0.0
        for day in hist[hist['Dividends'] > 0].index:
            price = hist.at[day, 'Close']
            div = hist.at[day, 'Dividends']
            hist.at[day, 'DividendYield'] = div / price if price else 0.0

        yearly = hist.resample('YE').agg({
            'Volume': 'mean',
            'MarketCap': 'mean',
            'DividendYield': 'mean'
        })

        yearly['Year'] = yearly.index.year
        yearly.reset_index(drop=True, inplace=True)
        yearly['Ticker'] = ticker

        volume_data.append(yearly[['Ticker', 'Year', 'Volume']])
        marketcap_data.append(yearly[['Ticker', 'Year', 'MarketCap']])
        dividendyield_data.append(yearly[['Ticker', 'Year', 'DividendYield']])

    except Exception as e:
        print(f"[ERROR] {ticker}: {e}")

for ticker in tickers:
    print(f"Processing: {ticker}")
    get_historical_data(ticker)
    time.sleep(1)

pd.concat(volume_data).to_csv(os.path.join(SAVE_DIR, 'Volume_Data.csv'), index=False)
pd.concat(marketcap_data).to_csv(os.path.join(SAVE_DIR, 'MarketCap_Data.csv'), index=False)
pd.concat(dividendyield_data).to_csv(os.path.join(SAVE_DIR, 'DividendYield_Data.csv'), index=False)

print("✅ Volume, MarketCap, and DividendYield data saved successfully.")

Processing: NVDA
Processing: MSFT
Processing: AAPL
Processing: AMZN
Processing: GOOGL
Processing: META
Processing: AVGO
Processing: TSLA
Processing: TSM
Processing: BRK-B
Processing: WMT
Processing: JPM
Processing: ORCL
Processing: V
Processing: LLY
Processing: MA
Processing: NFLX
Processing: XOM
Processing: COST
Processing: PLTR
Processing: JNJ
Processing: HD
Processing: PG
Processing: ABBV
Processing: SAP
Processing: BAC
Processing: CVX
Processing: KO
Processing: GE
Processing: AMD
Processing: CSCO
Processing: ASML
Processing: TMUS
Processing: BABA
Processing: PM
Processing: WFC
Processing: TM
Processing: IBM
Processing: CRM
Processing: ABT
Processing: AZN
Processing: MS
Processing: NVS
Processing: UNH
Processing: LIN
Processing: MCD
Processing: HSBC
Processing: NVO
Processing: INTU
Processing: SHEL
Processing: RTX
Processing: GS
Processing: BX
Processing: AXP
Processing: DIS
Processing: MRK
Processing: T
Processing: PEP
Processing: SHOP
Processing: CAT
Processing: UBER
Processing: R

$GEV: possibly delisted; no price data found  (1d 2010-01-01 -> 2023-12-31) (Yahoo error = "Data doesn't exist for startDate = 1262322000, endDate = 1703998800")


[ERROR] GEV: 'Dividends'
Processing: HDB
Processing: ANET
Processing: SCHW
Processing: BLK
Processing: BA
Processing: TMO
Processing: SPGI
Processing: TXN
Processing: ISRG
Processing: C
Processing: MUFG
Processing: PDD
Processing: QCOM
Processing: SONY
Processing: AMGN
Processing: BSX
Processing: ACN
Processing: UL
Processing: NEE
Processing: APP
Processing: TJX
Processing: AMAT
Processing: SYK
Processing: PGR
Processing: ARM
Processing: ADBE
Processing: DHR
Processing: SPOT
Processing: ETN
Processing: PFE
Processing: HON
Processing: DE
Processing: GILD
Processing: SAN
Processing: TTE
Processing: LOW
Processing: COF
Processing: APH
Processing: UNP
Processing: BHP
Processing: KKR
Processing: LRCX
Processing: TD
Processing: MU
Processing: UBS
Processing: ADP
Processing: BTI
Processing: KLAC
Processing: BUD
Processing: MELI
Processing: IBN
Processing: CMCSA
Processing: MDT
Processing: COP
Processing: DASH
Processing: SNPS
Processing: SNY
Processing: MSTR
Processing: WELL
Processing: PANW


HTTP Error 404: 
$APO-PRA: possibly delisted; no timezone found


[ERROR] APO-PRA: 'Dividends'
Processing: RACE
Processing: AON
Processing: SCCO
Processing: CVNA
Processing: MFG
Processing: ECL
Processing: EQIX
Processing: NEM
Processing: ITW
Processing: GSK
Processing: EMR
Processing: AJG
Processing: MSI
Processing: PNC
Processing: RSG
Processing: ABNB
Processing: NET
Processing: UPS
Processing: HWM
✅ Volume, MarketCap, and DividendYield data saved successfully.


In [4]:

TICKER_FILE_PATH = r"C:\Users\surji\Desktop\Quant_Poject\Complete_analysis_200stocks\Data\Top200_StockAnalysis.xlsx"
TICKER_COLUMN = "Ticker"
DATA_FOLDER = r"C:\Users\surji\Desktop\Quant_Poject\Complete_analysis_200stocks\Data\Files"
os.makedirs(DATA_FOLDER, exist_ok=True)

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Accept-Language': 'en-US,en;q=0.5',
    'Accept-Encoding': 'gzip, deflate',
    'Connection': 'keep-alive',
    'Upgrade-Insecure-Requests': '1',
}

In [5]:
def fetch_table(url):
    try:
        resp = requests.get(url, headers=headers, timeout=10)
        resp.raise_for_status()
        soup = bs4.BeautifulSoup(resp.text, "html.parser")
        tables = soup.find_all("table")
        if not tables:
            raise ValueError("No tables found on the page.")
        df = pd.read_html(StringIO(str(tables[0])))[0]
        if isinstance(df.columns, pd.MultiIndex):
            df.columns = df.columns.droplevel(0)
        df.columns = [col.strip() for col in df.columns] 
        return df
    except Exception as e:
        print(f"[ERROR] Failed to fetch data from {url}: {e}")
        return None



In [6]:

tickers_df = pd.read_excel(TICKER_FILE_PATH)
tickers_list = tickers_df[TICKER_COLUMN].dropna().unique().tolist()

metrics_info = {
    'PE': {
        'url_suffix': 'pe-ratio',
        'expected_cols': {'Date', 'Stock Price', 'TTM Net EPS', 'PE Ratio'},
        'rename_map': {'Date': 'Date', 'Stock Price': 'StockPrice', 'TTM Net EPS': 'NetEPS', 'PE Ratio': 'PE'},
        'save_file': 'pe_ratio_data.csv'
    },
    'PB': {
        'url_suffix': 'price-book',
        'expected_cols': {'Date', 'Stock Price', 'Book Value per Share', 'Price to Book Ratio'},
        'rename_map': {'Date': 'Date', 'Stock Price': 'StockPrice', 'Book Value per Share': 'BookValuePerShare', 'Price to Book Ratio': 'PB'},
        'save_file': 'pb_ratio_data.csv'
    },
    'ROE': {
        'url_suffix': 'roe',
        'expected_cols': {'Date', 'Return on Equity', 'TTM Net Income'},
        'rename_map': {'Date': 'Date', 'Return on Equity': 'ROE', 'TTM Net Income': 'NetIncome'},
        'save_file': 'roe_data.csv'
    },
    'ROA': {
        'url_suffix': 'roa',
        'expected_cols': {'Date', 'Return on Assets', 'Total Assets'},
        'rename_map': {'Date': 'Date', 'Return on Assets': 'ROA', 'Total Assets': 'TotalAssets'},
        'save_file': 'roa_data.csv'
    },
    'DebtToEquity': {
        'url_suffix': 'debt-equity-ratio',
        'expected_cols': {'Date', 'Debt to Equity Ratio'},
        'rename_map': {'Date': 'Date', 'Debt to Equity Ratio': 'DebtToEquity'},
        'save_file': 'debt_to_equity_data.csv'
    }
}


In [7]:
metric_data = {key: [] for key in metrics_info.keys()}

# Start scraping
for idx, ticker in enumerate(tickers_list):
    company_name = ticker.lower()
    print(f"Processing {idx+1}/{len(tickers_list)}: {ticker} ({company_name})")

    for metric_key, info in metrics_info.items():
        url = f"https://www.macrotrends.net/stocks/charts/{ticker}/{company_name}/{info['url_suffix']}"
        df_metric = fetch_table(url)

        if df_metric is not None and not df_metric.empty:
            if not info['expected_cols'].issubset(df_metric.columns):
                print(f"⚠️ Warning: Expected columns missing for {metric_key} of {ticker}")
            else:
                try:
                    df_metric = df_metric.rename(columns=info['rename_map'])
                    df_metric['Date'] = pd.to_datetime(df_metric['Date'], errors='coerce')
                    df_metric = df_metric[df_metric['Date'].between('2010-01-01', '2025-12-31')]

                    # Clean and convert metric columns to numeric
                    agg_columns = list(info['rename_map'].values())[1:]  # skip 'Date'
                    for col in agg_columns:
                        df_metric[col] = (
                            df_metric[col]
                            .astype(str)
                            .str.replace(',', '')
                            .str.replace('%', '')
                        )
                        df_metric[col] = pd.to_numeric(df_metric[col], errors='coerce')

                    df_metric['Ticker'] = ticker
                    df_metric['Year'] = df_metric['Date'].dt.year

                    # Aggregate quarterly → annual
                    df_annual = df_metric.groupby(['Ticker', 'Year'], as_index=False)[agg_columns].mean()

                    # Rename 'Year' → 'Date' and convert to datetime
                    df_annual = df_annual.rename(columns={'Year': 'Date'})
                    df_annual['Date'] = pd.to_datetime(df_annual['Date'], format='%Y')

                    metric_data[metric_key].append(df_annual)

                except Exception as e:
                    print(f"❌ Error processing {metric_key} data for {ticker}: {e}")
        else:
            print(f"⚠️ No data found for {metric_key} of {ticker}")

        time.sleep(random.uniform(15, 30))  # Respectful scraping delay

# Save all processed metrics
print("\n✅ All scraping complete. Saving files...\n")

for metric_key, df_list in metric_data.items():
    if df_list:
        combined_df = pd.concat(df_list, ignore_index=True)
        output_path = os.path.join(DATA_FOLDER, f"{metric_key}_Annual.csv")
        combined_df.to_csv(output_path, index=False)
        print(f"✅ Saved {metric_key} data to: {output_path}")
    else:
        print(f"⚠️ No data to save for {metric_key}")

Processing 1/200: NVDA (nvda)
Processing 2/200: MSFT (msft)
Processing 3/200: AAPL (aapl)
Processing 4/200: AMZN (amzn)
Processing 5/200: GOOGL (googl)
Processing 6/200: META (meta)
Processing 7/200: AVGO (avgo)
Processing 8/200: TSLA (tsla)
Processing 9/200: TSM (tsm)
Processing 10/200: BRK.B (brk.b)
Processing 11/200: WMT (wmt)
Processing 12/200: JPM (jpm)
Processing 13/200: ORCL (orcl)
Processing 14/200: V (v)
Processing 15/200: LLY (lly)
Processing 16/200: MA (ma)
Processing 17/200: NFLX (nflx)
[ERROR] Failed to fetch data from https://www.macrotrends.net/stocks/charts/NFLX/nflx/roa: HTTPSConnectionPool(host='www.macrotrends.net', port=443): Read timed out. (read timeout=10)
⚠️ No data found for ROA of NFLX
Processing 18/200: XOM (xom)
Processing 19/200: COST (cost)
Processing 20/200: PLTR (pltr)
Processing 21/200: JNJ (jnj)
[ERROR] Failed to fetch data from https://www.macrotrends.net/stocks/charts/JNJ/jnj/roe: HTTPSConnectionPool(host='www.macrotrends.net', port=443): Read timed 